In [1]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import re
import pandas as pd

In [2]:
ementa = list()
processo = list()
agencia_envolvida = list()
relator = list()
turma = list()
data_julgamento = list()
data_publicacao = list()
agencia_envolvida = list()

agencias = ['ANAC','ANEEL','ANTT', 'ANS', 'ANATEL', 'ANTAQ', 'ANVISA', 'ANP']
for agencia in agencias:
    path = 'D:/papers/direito/chromedriver_win32'
    final_path = 'D:/papers/direito/scrapping'

    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1240,1080") # define o tamanho da janela a ser aberta
    driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver
    url = 'https://scon.stj.jus.br/SCON/pesquisar.jsp'
    driver.get(url)
    
    pesquisa = driver.find_elements(by=By.XPATH, value='//*[@id="pesquisaLivre"]')
    #time.sleep(5)
    pesquisa[0].send_keys(f'"Recurso especial" {agencia} Nao embargo nao agravo') # buscando os acórdãos desejados

    pesquisa_botao = driver.find_element(by=By.XPATH, value='//*[@id="frmConsulta"]/div[3]/div[2]/div[2]/div/div/button') # salvando o botão de clique de busca
    #time.sleep(3)
    pesquisa_botao.click() # clicando na busca

    for pagina in range(20):
        
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página encontrada pela busca

        aumentar_acordaos = Select (driver.find_element(by=By.XPATH, value='//*[@id="qtdDocsPagina"]')) # expandindo o número de acórdãos por página
        aumentar_acordaos.select_by_value('50')
        
        simplificaco_botao = driver.find_element(by=By.XPATH, value='//*[@id="tp_vis_lista_resumida"]/span') # esse botão simplificará a página, facilitando a busca pelo texto da ementa
        #time.sleep(3)
        simplificaco_botao.click()
    
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página simplificada

        for i in range(0,len(soup.find_all('div',{'class':'row itemlistadocumentos p-2'}))): # obtendo a ementa
            ementa.append(soup.find_all('div', {'class':'clsEmentaCompleta'})[i].get_text().replace('\n','').replace('\t',''))

        completo_botao = driver.find_element(by=By.XPATH, value='//*[@id="tp_vis_documento_completo"]/span') # esse botão volta a página à sua forma inicial, facilitando a busca por outros elementos
        #time.sleep(3)
        completo_botao.click()

        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página completa

        for i in range(0, len(soup.find_all('div',{'class':'documento'}))): 
            agencia_envolvida.append(agencia) # salvando a agência que está sendo buscada
            
        for i in range(0,len(soup.find_all('div',{'class':'col-md-6 col-sm-12'})),2):
            processo.append(re.search('\d{3,}\s\/\s\w\w', soup.find_all('div',{'class':'col-md-6 col-sm-12'})[i].get_text())[0]) # salvando o elemento com detalhes de identificação do processo

        for i in range(0,len(soup.find_all('div',{'class':'col-md-3 col-sm-12'})),4): # esse elemento é uma lista de quatro elementos semelhantes, então o contador pulará de quatro em quatro
            relator.append(re.subn('Relator[a]?', '', re.subn('\\n','',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[0].get_text())[0])[0]) # salvando o relator
            turma.append(re.search('(?<=\\n\\nÓrgão Julgador\\n).*(?=\\n\\n)',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 1].get_text())[0]) # salvando a turma
            data_julgamento.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 2].get_text())[0]) # salvando a data
            data_publicacao.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 3].get_text())[0]) # salvando a data
        
        try:
            if pagina == 0:
                #time.sleep(3)
                driver.find_elements(by=By.XPATH, value='//*[@id="navegacao"]/div[2]/a[1]/span')[0].click() # clique para a próxima página quando se está na primeira página
            else:
                #time.sleep(3)
                driver.find_elements(by=By.XPATH, value='//*[@id="navegacao"]/div[2]/a[3]/span')[0].click() # clique para a próxima página quando não se está na primeira página
        except:
            break

    driver.quit()

dados1 = {'processo': processo,
    'ministro_relator': relator,
    'orgao_julgador': turma,
    'agencia': agencia_envolvida,
    'data_julgamento': data_julgamento,
    'data_publicacao': data_publicacao,
    'ementa': ementa
} # salvando todas as listas obtidas num dicionário

df1 = pd.DataFrame.from_dict(dados1).drop_duplicates() # transformando o dicionário em um dataframe
df1.to_excel('D:/papers/direito/scrapping/base.xlsx',index=False)
df1

C:\Users\alves\AppData\Local\Temp/ipykernel_14172/3494283118.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver


,processo,ministro_relator,orgao_julgador,agencia,data_julgamento,data_publicacao,ementa
0,1611915 / RS,Ministro MARCO BUZZI (1149),T4 - QUARTA TURMA,ANAC,06/12/2018,04/02/2019,RECURSO ESPECIAL - AÇÃO CONDENATÓRIA - ACESSIB...
1,1769070 / SP,Ministro MARCO BUZZI (1149),T2 - SEGUNDA TURMA,ANAC,13/11/2018,21/11/2018,PROCESSUAL CIVIL E TRIBUTÁRIO. EXECUÇÃO FISCAL...
2,1726216 / RJ,Ministro MARCO BUZZI (1149),T2 - SEGUNDA TURMA,ANAC,15/05/2018,21/11/2018,DIREITO DO CONSUMIDOR. ESPAÇO ENTRE AS POLTRON...
3,1316372 / DF,Ministro MARCO BUZZI (1149),T2 - SEGUNDA TURMA,ANAC,06/12/2016,19/12/2016,PROCESSUAL CIVIL. AÇÃO CAUTELAR DE PRODUÇÃO AN...
4,1469087 / AC,Ministro MARCO BUZZI (1149),T2 - SEGUNDA TURMA,ANAC,18/08/2016,17/11/2016,CONSUMIDOR. CONCESSÃO DE SERVIÇOS AÉREOS. RELA...
...,...,...,...,...,...,...,...
295,1074584 / RJ,Ministro RICARDO VILLAS BÔAS CUEVA (1147),T1 - PRIMEIRA TURMA,ANS,09/06/2009,24/06/2009,TRIBUTÁRIO E PROCESSO CIVIL. NEGATIVA DE PREST...
296,963531 / RJ,Ministro RICARDO VILLAS BÔAS CUEVA (1147),T1 - PRIMEIRA TURMA,ANS,02/06/2009,10/06/2009,MANDADO DE SEGURANÇA. TAXA DE SAÚDE SUPLEMENTA...
297,728330 / RJ,Ministra DENISE ARRUDA (1126),T1 - PRIMEIRA TURMA,ANS,19/03/2009,15/04/2009,PROCESSUAL CIVIL E TRIBUTÁRIO. ALEGADA VIOLAÇÃ...
298,892932 / RJ,Ministra DENISE ARRUDA (1126),T2 - SEGUNDA TURMA,ANS,09/12/2008,27/02/2009,PROCESSUAL CIVIL - LITISPENDÊNCIA - AUSÊNCIA D...
